In [3]:
import numpy as np
import scipy.sparse as sp
import sparse
import torch
import scipy.signal
import numba
import pyfftw
pyfftw.interfaces.cache.enable()

In [4]:
# 1 MiB tile size
size = 1024*1024*1

(fy, fx) = (512, 512)

l = size // (fy*fx)

print(l)

data = pyfftw.zeros_aligned((l, fy*fx))

data[:] = np.random.random((l, fy*fx))

data_t = data.T.copy()

n_masks = 1000
n_entries = 20

masks = pyfftw.zeros_aligned((n_masks, (fy*fx)))

mask_indices = []
frame_indices = []
values = []

for i in range(n_masks):
    for y in np.random.choice(range(fy), size=n_entries, replace=False):
        for x in np.random.choice(range(fx), size=n_entries, replace=False):
            index = y*fx + x
            mask_indices.append(i)
            frame_indices.append(index)
            masks[i, index] = 1
            values.append(1)

4


In [5]:
@numba.njit
def _dot(values, iis, data, res):
    # Magic number 32: This is the smallest number where overheads
    # did not have an impact on the performance
    j_block_size = 32
    j_blocks = data.shape[0] // j_block_size
    j_remainder = data.shape[0] % j_block_size
    # The blocking helps to keep iis and values in the cache for
    # the j range that is being processed
    for j_block in range(j_blocks):
        for idx in range(values.shape[0]):
            # This is also values.shape[1]
            for j in range(j_block*j_block_size, (j_block + 1)*j_block_size):
                i = iis[idx, j]
                v = values[idx, j]
                for k in range(data.shape[1]):
                    # FIXME sum is not numerically stable
                    # That should be tolerable if the matrix is sparse, i.e. there aren't many summands
                    # Stable implementation may require an intermediate buffer
                    res[i, k] += data[j, k] * v
    for idx in range(values.shape[0]):
        # This is also values.shape[1]
        for j in range(j_blocks*j_block_size, j_blocks*j_block_size + j_remainder):
            i = iis[idx, j]
            v = values[idx, j]
            for k in range(data.shape[1]):
                # FIXME sum is not numerically stable
                # That should be tolerable if the matrix is sparse, i.e. there aren't many summands
                # Stable implementation may require an intermediate buffer
                res[i, k] += data[j, k] * v
                
@numba.njit
def _transposed_left_dot(data, values, iis, res):
    '''
    This function performs dot(data, masks.T) with an optimized
    access pattern. This matches the way how data and mask are "naturally"
    stored and handled in LiberTEM
    '''
    # Magic number 32: This is the smallest number where overheads
    # did not have an impact on the performance
    j_block_size = 32
    j_blocks = data.shape[1] // j_block_size
    j_remainder = data.shape[1] % j_block_size
    
    # The blocking helps to keep iis and values in the cache for
    # the j range that is being processed
    for j_block in range(j_blocks):
        for idx in range(values.shape[0]):
            for j in range(j_block*j_block_size, (j_block + 1)*j_block_size):
                i = iis[idx, j]
                v = values[idx, j]
                for k in range(data.shape[0]):
                    # FIXME sum is not numerically stable
                    # That should be tolerable if the matrix is sparse, i.e. there aren't many summands
                    # Stable implementation may require an intermediate buffer
                    res[k, i] += data[k, j] * v
    for idx in range(values.shape[0]):
        for j in range(j_blocks*j_block_size, j_blocks*j_block_size + j_remainder):
            i = iis[idx, j]
            v = values[idx, j]
            for k in range(data.shape[0]):
                # FIXME sum is not numerically stable
                # That should be tolerable if the matrix is sparse, i.e. there aren't many summands
                # Stable implementation may require an intermediate buffer
                res[k, i] += data[k, j] * v

# Necessary for inlining in Nopython mode
@numba.njit
def _add_index_depth(values, iis, n):
    iis = np.concatenate((iis, np.zeros((n, iis.shape[1]), dtype=iis.dtype)), axis=0)
    values = np.concatenate((values, np.zeros((n, values.shape[1]), dtype=values.dtype)), axis=0)
    return (values, iis)
                
@numba.njit                                
def _set_coords(new_iis, new_jjs, new_vals, indices, iis, values):
    for k in range(len(new_iis)):
        i = new_iis[k]
        j = new_jjs[k]
        idx = 0
        new = True
        for idx in range(indices[j]):
            if iis[idx, j] == i and values[idx, j] != 0:
                new = False
                break
        # was unset and remains unset
        if new and new_vals[k] == 0:
            continue
        if new:
            idx += 1
        if values.shape[0] <= idx:
            (values, iis) = _add_index_depth(values, iis, n=idx - values.shape[0] + 1)

        iis[idx, j] = i
        values[idx, j] = new_vals[k]
        if new:
            indices[j] = idx + 1
    return (indices, iis, values)


@numba.njit
def _todense(shape, iis, values):
    res = np.zeros(shape=shape, dtype=values.dtype)
    for j in range(iis.shape[1]):
        for idx in range(iis.shape[0]):
            i = iis[idx, j]
            v = values[idx, j]
            if v != 0:
                res[i, j] = v
    return res

class StackMatrix(object):
    
    def __init__(self, shape, dtype, indices, values, iis):
        self._shape = shape
        self._dtype = dtype
        self._indices = indices
        self._values = values
        self._iis = iis
    
    @classmethod
    def from_numpy(cls, matrix):
        '''
        For simplicity, only support m x n matrices for now
        '''
        assert len(matrix.shape) == 2        
        shape = matrix.shape
        dtype = matrix.dtype
        (i, j) = np.mgrid[0:matrix.shape[0], 0:matrix.shape[1]]
        non_zero = (matrix != 0)
        depth = np.max(non_zero.astype(np.int64).sum(axis=0))

        m = cls.zeros(shape=shape, dtype=dtype, depth=depth)
        m.set_coords(i[non_zero], j[non_zero], matrix[non_zero])              
        return m
    
    @classmethod
    def from_sparse(cls, sp):
        non_zero = (sp != 0)
        depth = np.max(non_zero.astype(np.int64).sum(axis=0))
        m = cls.zeros(shape=shape, dtype=dtype, depth=depth)
        m.set_coords(*sp.coords, vals=sp.data)
                    
    @classmethod
    def zeros(cls, shape, dtype=np.float64, depth=0):
        assert len(shape) == 2
        dtype = np.dtype(dtype)
        
        indices = np.zeros(shape[1], dtype=np.int64)
        values = np.zeros((depth, shape[1]), dtype=dtype)
        iis = np.zeros((depth, shape[1]), dtype=np.int64)
        
        return cls(shape, dtype, indices, values, iis)
                    
    def __getitem__(self, idx):
        i, j = idx
        idx = np.where(self._iis[:, j] == i)
        if idx:
            return self._values[idx, j]
        else:
            return 0
        
    def set_layer(self, i, mask):
        non_zero = mask != 0
        jj = np.arange(self._shape[1], dtype=np.int64)
        ii = i*np.ones(self._shape[1], dtype=np.int64)
        self.set_coords(iis=ii[non_zero], jjs=jj[non_zero], vals=mask[non_zero])
    
    def set_coords(self, iis, jjs, vals):
        (self._indices, self._iis, self._values) = _set_coords(
            new_iis=iis, new_jjs=jjs, new_vals=vals,
            indices=self._indices, iis=self._iis, values=self._values
        )        
    
    def __setitem__(self, idx, value):
        i, j = idx
        self.set_coords([i], [j], [value])

    def add_index_depth(self, n=1):
        (self._values, self._iis) = _add_index_depth(self._values, self._iis, n)

    def dot(self, data):
        assert data.shape[0] == self._shape[1]
        res = np.zeros((self._shape[0], data.shape[1]), dtype=np.float64)
        _dot(values=self._values, iis=self._iis, data=data, res=res)
        return res
    
    def transposed_left_dot(self, data):
        assert data.shape[1] == self._shape[1]
        res = np.zeros((data.shape[0], self._shape[0]), dtype=np.float64)
        _transposed_left_dot(data=data, values=self._values, iis=self._iis, res=res)
        return res
    
    def todense(self):
        return _todense(self._shape, self._iis, self._values)
    
    def tosparse(self):
        nonzero = self._values != 0
        iis = self._iis[nonzero]
        row = np.arange(self._shape[1], dtype=np.int64)
        jjs = np.tile(row, (self._iis.shape[0], 1))[nonzero]
        return sparse.COO(coords=(iis, jjs), data=self._values[nonzero], shape=self._shape)
    
    def tocsc(self):
        nonzero = self._values != 0
        iis = self._iis[nonzero]
        row = np.arange(self._shape[1], dtype=np.int64)
        jjs = np.tile(row, (self._iis.shape[0], 1))[nonzero]
        return scipy.sparse.csc_matrix((self._values[nonzero], (iis, jjs)), shape=self._shape)

    @property
    def shape(self):
        return self._shape
    
    @property
    def dtype(self):
        return self._dtype
    
    @property 
    def depth(self):
        return self._iis.shape[0]

In [6]:
# naive one
d = np.dot(data, masks.T)
%timeit np.dot(data, masks.T)

172 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# naive one
%timeit np.dot(masks, data_t)

173 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
homebuilt = StackMatrix.from_numpy(masks)
homebuilt_t = StackMatrix.from_numpy(masks.T)

In [9]:
print(data_t.shape)
print(data.shape)
print(homebuilt.shape)
print(homebuilt_t.shape)

(262144, 4)
(4, 262144)
(1000, 262144)
(262144, 1000)


In [10]:
homebuilt.dot(data_t)
%timeit homebuilt.dot(data_t)

21.3 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
homebuilt.transposed_left_dot(data)
%timeit homebuilt.transposed_left_dot(data)

23.8 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
t_data = torch.from_numpy(data)
t_masks = torch.from_numpy(masks.T)
torch.mm(t_data, t_masks)
%timeit torch.mm(t_data, t_masks)

117 ms ± 985 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
sp_csr_masks = sp.csr_matrix(masks)
sp_csr_masks.dot(data_t)
%timeit sp_csr_masks.dot(data_t)

4.74 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
sp_csr_array = sp.csr_array(masks)
sp_csr_array.dot(data_t)
%timeit sp_csr_array.dot(data_t)

4.78 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
sp_csc_masks = sp.csc_matrix(masks)
sp_csc_masks.dot(data_t)
%timeit sp_csc_masks.dot(data_t)

4.05 ms ± 50.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
sp_csr_masks_t = sp.csr_matrix(masks.T)
tmp = data * sp_csr_masks_t
print(np.allclose(d, tmp))
%timeit data * sp_csr_masks_t

True
8.18 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
sp_csc_masks_t = sp.csc_matrix(masks.T)
data * sp_csc_masks_t
%timeit data * sp_csc_masks_t

10.2 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
i = torch.LongTensor([mask_indices, frame_indices])
v = torch.DoubleTensor(values)
t_sp_mask = torch.sparse.DoubleTensor(i, v, torch.Size([n_masks, fy*fx]))
t_sp_data = torch.from_numpy(data.T)
torch.sparse.mm(t_sp_mask, t_sp_data)
%timeit torch.sparse.mm(t_sp_mask, t_sp_data)

27.1 ms ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
pydata_sp_mask = sparse.COO([frame_indices, mask_indices], values, shape=(fy*fx, n_masks))
sparse.dot(data, pydata_sp_mask)
%timeit sparse.dot(data, pydata_sp_mask)

94 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
pydata_sp_mask_t = sparse.COO([frame_indices, mask_indices], values, shape=(fy*fx, n_masks)).T.copy()
pydata_sp_mask_t @ data_t
%timeit pydata_sp_mask_t @ data_t

33.4 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
pydata_sp_gcxs = sparse.GCXS.from_coo(pydata_sp_mask)
sparse.dot(data, pydata_sp_gcxs)
%timeit sparse.dot(data, pydata_sp_gcxs)

6.04 ms ± 353 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
pydata_sp_gcxs_t = sparse.GCXS.from_coo(pydata_sp_mask_t)
pydata_sp_gcxs_t @ data_t
%timeit pydata_sp_gcxs_t @ data_t

7.97 ms ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
fft = pyfftw.interfaces.numpy_fft
# fft = np.fft

In [18]:
m = masks[0].reshape((fy, fx))
template = pyfftw.interfaces.numpy_fft.rfft2(m)
%timeit template = pyfftw.interfaces.numpy_fft.rfft2(m)

5.18 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
for d in data.reshape((-1, fy, fx)):
    spec = pyfftw.interfaces.numpy_fft.rfft2(d)
    corrspec = spec * template
    corr = pyfftw.interfaces.numpy_fft.fftshift(pyfftw.interfaces.numpy_fft.irfft2(corrspec))

69 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit
for d in data.reshape((-1, fy, fx)):
    spec = np.fft.rfft2(d)
    corrspec = spec * template
    corr = np.fft.fftshift(np.fft.irfft2(corrspec))

187 ms ± 5.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
for d in data.reshape((-1, fy, fx)):
    corr = scipy.signal.convolve(m, d)

936 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
scipy.fft = pyfftw.interfaces.scipy_fftpack
scipy.fftpack = pyfftw.interfaces.scipy_fftpack

In [23]:
%%timeit
for d in data.reshape((-1, fy, fx)):
    corr = scipy.signal.convolve(m, d)

942 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
